# DEMO notebook for GC-Spectro probe

This notebook stands as a demonstration for the GC-Spectro probe, i.e. the multipoles $P_{\rm gg}^{(\ell)}(k,z)$ of the anisotropic galaxy power spectrum $P_{\rm gg}(k,\mu,z)$, which are defined as
$$P_{\rm gg}^{(\ell)}(k,z) = \frac{2\ell+1}{2}\int_{-1}^{+1}{\rm d}\mu'\,P_{\rm gg}(k',\mu',z)\,{\cal L}_{\ell}(\mu')\,,$$
where ${\cal L}_{\ell}(\mu)$ is the $\ell$-th order Legendre polynomial, and Alcock-Paczynski corrections are properly included via the rescaling of the wavemode $k\rightarrow k'$ and the pair orientation $\mu\rightarrow \mu'$.\
The notebook shows how to obtain both linear and nonlinear predictions; in the latter case the only available model so far is the [EFT model](https://arxiv.org/abs/1909.05277).

### 1) Importing external modules

First, we load the external modules needed to run the notebook:\
**numpy** --> for mathematical operations and handling of arrays,\
**time** --> for measuring time performances,\
**os, sys** --> for handling filenames, paths, and directories,\
**copy** --> for creating copies of Python objects

In [ ]:
import numpy as np
import time 
import os, sys
import copy

We also import **matplotlib**, and setup the default options for plotting.

In [ ]:
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

plt.rc('xtick', labelsize=15, direction='in')
plt.rc('ytick', labelsize=15, direction='in')
plt.rc('font', size=15)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=15)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=15)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

If you have cloned the repository, the location of this notebook should be in `likelihood-implementation/notebooks/`.\
Let's set the working directory to be `likelihood-implementation/`.

In [ ]:
likelihood_path = os.path.realpath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory:', likelihood_path)
print ('Conda environment:', os.environ['CONDA_DEFAULT_ENV'])

### 2) Setup of the call to Cobaya

We also import the `EuclidLikelihood` class, that contains the external likelihood of CLOE. This is stored within the module `cloe/cobaya_interface.py`.

In [ ]:
from cloe.cobaya_interface import EuclidLikelihood

We are specifying this likelihood as an external likelihood class to be passed to Cobaya. In this case, Cobaya requires either a dictionary or a yaml file containing all the information necessary to run. Here, we set up the info dictionary. Since we are only interested in running GC-Spectro, all the nuisance parameters that refer to the 3x2pt observables are not used in the likelihood. However, we still need to specify them since we still don't have a switch to turn off the computation of unrequested probes.

In [ ]:
info = {
    # 'params': Cobaya's protected key of the input dictionary. 
    # Includes the parameters that the user would like to sample over:
    'params': {
        ###########################
        # Cosmological parameters #
        ###########################
        'tau': 0.0925,      # Optical depth
        'H0': 67.0,         # Hubble parameter at z=0 [km/s/Mpc]
        'omk': 0.0,         # Fractional curvature density
        'ombh2': 0.022445,  # Physical baryon density
        'omch2': 0.1205579, # Physical cold dark matter density
        'mnu': 0.06,        # Sum of individual neutrino masses [eV]
        'nnu': 3.046,       # Effective number of relativistic species
        'w': -1,            # Dark energy equation of state at z=0
        'wa': 0,            # Time-evolution of the dark energy equation of state (CLP parametrization)
        'As': 2.12605e-9,   # Scalar amplitude of the primordial power spectrum
        'ns': 0.96,         # Scalar index of the primordial power spectrum
        'omegam': None,     # DERIVED parameter: fractional matter density
        'omegab': None,     # DERIVED parameter: fractional baryon density
        'omeganu': None,    # DERIVED parameter: fractional neutrino density
        'omnuh2': None,     # DERIVED parameter: physical neutrino density
        'omegac': None,     # DERIVED parameter: fractional cold dark matter density
        'N_eff': None,      # DERIVED parameter: effective number of relativistic species
        ##########################
        # Photometric parameters #
        ##########################
        'b1_photo': 1.0997727037892875, 'b2_photo': 1.220245876862528, 'b3_photo': 1.2723993083933989, 'b4_photo': 1.316624471897739, 'b5_photo': 1.35812370570578,
        'b6_photo': 1.3998214171814918, 'b7_photo': 1.4446452851824907, 'b8_photo': 1.4964959071110084, 'b9_photo': 1.5652475842498528, 'b10_photo': 1.7429859437184225,
        # Magnification bias parameters
        'magnification_bias_1': 0.0, 'magnification_bias_2': 0.0, 'magnification_bias_3': 0.0, 'magnification_bias_4': 0.0, 'magnification_bias_5': 0.0,
        'magnification_bias_6': 0.0, 'magnification_bias_7': 0.0, 'magnification_bias_8': 0.0, 'magnification_bias_9': 0.0, 'magnification_bias_10': 0.0,
        # Shear calibration multiplicative bias parameters                                                                                                                                                                                                                                                                                                            
        'multiplicative_bias_1': 0.0, 'multiplicative_bias_2': 0.0, 'multiplicative_bias_3': 0.0, 'multiplicative_bias_4': 0.0, 'multiplicative_bias_5': 0.0,
        'multiplicative_bias_6': 0.0, 'multiplicative_bias_7': 0.0, 'multiplicative_bias_8': 0.0, 'multiplicative_bias_9': 0.0, 'multiplicative_bias_10': 0.0,
        # Intrinsic alignment parameters
        'aia': 1.72,
        'nia': -0.41,
        'bia': 0.0,
        # Redshift distributions nuisance parameters: shifts
        'dz_1_GCphot': 0., 'dz_1_WL': 0., 'dz_2_GCphot': 0., 'dz_2_WL': 0., 'dz_3_GCphot': 0., 'dz_3_WL': 0., 'dz_4_GCphot': 0., 'dz_4_WL': 0., 'dz_5_GCphot': 0., 'dz_5_WL': 0.,
        'dz_6_GCphot': 0., 'dz_6_WL': 0., 'dz_7_GCphot': 0., 'dz_7_WL': 0., 'dz_8_GCphot': 0., 'dz_8_WL': 0., 'dz_9_GCphot': 0., 'dz_9_WL': 0., 'dz_10_GCphot': 0., 'dz_10_WL': 0.,
        ############################
        # Spectroscopic parameters #
        ############################
        # Linear bias #
        'b1_spectro_bin1': 1.46, 'b1_spectro_bin2': 1.61, 'b1_spectro_bin3': 1.75, 'b1_spectro_bin4': 1.90,
        # Local quadratic bias #
        'b2_spectro_bin1': 0.0, 'b2_spectro_bin2': 0.0, 'b2_spectro_bin3': 0.0, 'b2_spectro_bin4': 0.0,
        # FOG counterterms #
        'c0_spectro_bin1': 0.0, 'c0_spectro_bin2': 0.0, 'c0_spectro_bin3': 0.0, 'c0_spectro_bin4': 0.0,
        'c2_spectro_bin1': 0.0, 'c2_spectro_bin2': 0.0, 'c2_spectro_bin3': 0.0, 'c2_spectro_bin4': 0.0,
        'c4_spectro_bin1': 0.0, 'c4_spectro_bin2': 0.0, 'c4_spectro_bin3': 0.0, 'c4_spectro_bin4': 0.0,
        # Deviations from Poisson shot-noise #
        'aP_spectro_bin1': 0.0, 'aP_spectro_bin2': 0.0, 'aP_spectro_bin3': 0.0, 'aP_spectro_bin4': 0.0,
        # Inverse number density #
        'Psn_spectro_bin1': 0.0, 'Psn_spectro_bin2': 0.0, 'Psn_spectro_bin3': 0.0, 'Psn_spectro_bin4': 0.0,
        # Growth index
        'gamma_MG': 0.55
    },
    # 'likelihood': Cobaya's protected key of the input dictionary.
    # Set up of the run, choice of data samples, flags, etc.
    'likelihood': {
        'Euclid': {
            # Likelihood Class to be read as external
            'external': EuclidLikelihood,
            # Select which observables to use during the analysis (here only GCspectro)
            'observables_selection': {
                'WL': {
                    'WL': False, 'GCphot': False, 'GCspectro': False
                },
                'GCphot': {
                    'GCphot': False, 'GCspectro': False
                },
                'GCspectro': {
                    'GCspectro': True
                }
            },         
            # Plot the selected observables matrx
            'plot_observables_selection': True,  
            # Nonlinear flags to select which nonlinear model to adopt
            # Available options are:
            # a) NL_flag_phot_matter (nonlinear model of matter power spectrum for photometric probes)
            #    0 -> linear theory
            #    1 -> Takahashi
            #    2 -> Mead2020 (w/o baryon corrections)
            #    3 -> EE2
            #    4 -> Bacco (matter)
            # b) NL_flag_spectro (nonlinear model of galaxy power spectrum for GCspectro)
            #    0 -> linear theory (linear matter evolution, linear bias, and Kaiser factor)
            #    1 -> EFT
            'NL_flag_phot_matter': 0,
            'NL_flag_spectro': 0,
            # 'data': This give specifications for the paths of the input data files
            'data': { 
                # 'sample' specifies the first folder below the main data folder
                'sample': 'ExternalBenchmark',
                # 'spectro' and 'photo' specify paths to data files.
                'spectro': {
                    # GC Spectro root name should contain z{:s} string to enable iteration over bins
                    'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                    'redshifts': ["1.", "1.2", "1.4", "1.65"],
                    'edges': [0.9, 1.1, 1.3, 1.5, 1.8]
                },
                'photo': {
                    'ndens_GC': 'niTab-EP10-RB00.dat',
                    'ndens_WL': 'niTab-EP10-RB00.dat',
                    'luminosity_ratio': 'luminosity_ratio.dat',
                    # Photometric root names should contain z{:s} string to specify IA model
                    'root_GC': 'Cls_{:s}_PosPos.dat',
                    'root_WL': 'Cls_{:s}_ShearShear.dat',
                    'root_XC': 'Cls_{:s}_PosShear.dat',
                    'IA_model': 'zNLA',
                    # Photometric covariances root names should contain z{:s} string to specify how the covariance was calculated
                    'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npy',
                    'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npy',
                    'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npy',
                    'cov_model': 'Gauss'
                }
            }
        }
    },
    # 'theory': Cobaya's protected key of the input dictionary.
    # Minimum theoretical requirements to a Boltzman Solver (CAMB in this case)
    'theory': {
        'camb': {
            'stop_at_error': True, 
            'extra_args': {
                'num_massive_neutrinos': 1,
                'dark_energy_model': 'ppf'
            }
        }
    },
    # 'sampler': Cobaya's protected key of the input dictionary.
    # In this DEMO, we use the 'evaluate' sampler to make a single computation of the posterior distributions
    'sampler': {
        'evaluate': None
    },  
    # 'output': Cobaya's protected key of the input dictionary.
    # Defines path for output products
    'output': 'chains/my_euclid_experiment',
    # 'debug': Cobaya's protected key of the input dictionary.
    # Level of detail of debug
    'debug': False,
    # 'timing': Cobaya's protected key of the input dictionary.
    # Decides whether to print the time to compute the posterior distribution or not
    'timing': True,
    # 'force': Cobaya's protected key of the input dictionary.
    # If True, Cobaya forces deleting the previous output files with the same name
    'force': True,
}

We need to call this method to update the `params.yaml` file that contains all the nuisance parameters of CLOE that would be otherwise unkwown to COBAYA. Here the update is based on the info dictionary defined above.

In [ ]:
from cloe.auxiliary.likelihood_yaml_handler import write_params_yaml_from_info_dict
write_params_yaml_from_info_dict(info)

Now that the `params.yaml` is updated, we can pass the info dictionary to Cobaya. Since we want to access the internal quantities of our likelihood module, we are going to make use of the `model` object from Cobaya. From this object it is later possible to get all the information we want.\
Before that, we are setting the nonlinear model for the matter power spectrum coming from the Boltzmann solver according to the value of the nonlinear flag for the photometric probes, `NL_flag_photo_matter`. In this case, this is not necessary, since we are not using the photometric probes.

In [ ]:
from cloe.auxiliary.likelihood_yaml_handler import set_halofit_version
set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'])

from cobaya.model import get_model

The `get_model` function of Cobaya imported in the line above needs a yaml or dictionary as an argument, so we pass the one that we have defined in the previous cell. This function is going to initialize the `EuclidLikelihood` class. However, in order to have an updated instance of the class, in which all the requirements have been properly received from the Boltzmann solver, we need to make an evaluation of the likelihood.

In [ ]:
t1 = time.time()
model = get_model(info)
print('Time for initialization of the likelihood: ', time.time()-t1)

t1 = time.time()
logposterior = model.logposterior({})
print('Time for evaluation of the likelihood: ', time.time()-t1)

**Please note:** even though we are not asking for the 3x2pt photomteric observables, the current version of CLOE initialises also the `Photo` class (the one dedicated to the calculations for the 3x2pt statistics), and therefore part of the time required for the evaluation of the likelihood is due to this extra time. This time is added to the evaluation of the likelihood and not to its initialization, since everything happens in the `update` method of the `Photo` class, which is called only when evaluating the likelihood.

Finally, we create an instance of `EuclidLikelihood`, and we pass to it all the attributes from the `model` wrapper defined in the previous cell. In this way, we are now allowed to access everything that is an attribute the `EuclidLikelihood` class.

In [ ]:
like = EuclidLikelihood()
like.initialize()
like.passing_requirements(model, info, **model.provider.params)

### 3) Plotting the spectroscopic galaxy power spectrum $P_{\rm gg}(k,\mu,z)$ (i.e. internal interpolator of the `Cosmology` class).

Here we set the arrays of wavelengths $k$, pair orientations $\mu$, and redshifts $z$ used for the plots.

In [ ]:
ks = np.logspace(-3, 1, 200)
mus = np.linspace(0.0, 1.0, 8)
z = 1.0

In order for the internal interpolators of CLOE to be updated, we need to run the `update_cosmo_dic` method of the `Cosmology` class. This will setup all the interpolators according to the current flags and parameters of the cosmo dictionary. In addition we save a deep copy of the cosmo dictionaries after the interpolators have been redefined, so that we can use them later to create instances of the `Spectro` class.

The first case we want corresponds to the original linear theory predictions, and therefore `NL_flag_spectro` is set to 0.

In [ ]:
like.cosmo.cosmo_dic['NL_flag_spectro'] = 0
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)
cosmo_dic_0 = copy.deepcopy(like.cosmo.cosmo_dic)

The second case corresponds to the EFT model but with all the nuisance parameters set to 0. In this case, therefore, `NL_flag_spectro` is set to 1, and we manually set the extra EFT parameters to 0 (they are already set to 0, from the definition of the info dictionary, but we do it again in case this cell is run ahgain after the next one).

In [ ]:
like.cosmo.cosmo_dic['NL_flag_spectro'] = 1
like.cosmo.cosmo_dic['nuisance_parameters']['b2_spectro_bin1'] = 0.0
like.cosmo.cosmo_dic['nuisance_parameters']['c0_spectro_bin1'] = 0.0
like.cosmo.cosmo_dic['nuisance_parameters']['c2_spectro_bin1'] = 0.0
like.cosmo.cosmo_dic['nuisance_parameters']['c4_spectro_bin1'] = 0.0
like.cosmo.cosmo_dic['nuisance_parameters']['aP_spectro_bin1'] = 0.0
like.cosmo.cosmo_dic['nuisance_parameters']['Psn_spectro_bin1'] = 0.0
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)
cosmo_dic_1 = copy.deepcopy(like.cosmo.cosmo_dic)

The third case corresponds once again to the EFT model, but with the extra nuisance parameters no longer set to 0. This is going to show that the interpolator for $P_{\rm gg}(k,\mu,z)$ correctly sees modifications in the values of these parameters. The choice of the parameters here is arbitrary. We only provide an example for the first redshift bin, but this can be easily extended to other redshifts simply by modifying the corresponding nuisance parameters.

In [ ]:
# Here we generate a cosmo dictionary with EFT with some values for the nuisance parameters
like.cosmo.cosmo_dic['NL_flag_spectro'] = 1
like.cosmo.cosmo_dic['nuisance_parameters']['b2_spectro_bin1'] = -0.2
like.cosmo.cosmo_dic['nuisance_parameters']['c0_spectro_bin1'] = 1.0
like.cosmo.cosmo_dic['nuisance_parameters']['c2_spectro_bin1'] = 1.0
like.cosmo.cosmo_dic['nuisance_parameters']['c4_spectro_bin1'] = 1.0
like.cosmo.cosmo_dic['nuisance_parameters']['aP_spectro_bin1'] = 0.5
like.cosmo.cosmo_dic['nuisance_parameters']['Psn_spectro_bin1'] = 1000.0
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)
cosmo_dic_2 = copy.deepcopy(like.cosmo.cosmo_dic)

The EFT model still features deviations from linear theory predictions even with all the higher-order parameters set to zero. This happens because there are extra one-loop contributions to the power spectrum that comes from the auto-correlation of the velocity divergence field with itself, and these are quantities that are not rescaled by a bias parameter. Let's see it directly in the plots.

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(11,5))

for i in range(len(mus)):
    
    ind1 = int(np.floor(i/4.0))
    ind2 = i-ind1*4
    ax = axs[ind1,ind2]
    
    mu = mus[i]
    
    ax.grid(ls='--', lw=0.3)

    if ind1==0 and ind2==0:
        ax.loglog(ks, cosmo_dic_0['Pgg_spectro'](z, ks, mu), lw=3, label='Linear theory')
        ax.loglog(ks, cosmo_dic_1['Pgg_spectro'](z, ks, mu), lw=3, label='EFT (all params = 0)')
        ax.loglog(ks, cosmo_dic_2['Pgg_spectro'](z, ks, mu), lw=3, label='EFT')
    else:
        ax.loglog(ks, cosmo_dic_0['Pgg_spectro'](z, ks, mu), lw=3)
        ax.loglog(ks, cosmo_dic_1['Pgg_spectro'](z, ks, mu), lw=3)
        ax.loglog(ks, cosmo_dic_2['Pgg_spectro'](z, ks, mu), lw=3)

    ax.set_ylim([0.1, 3e5])
    
    if ind2==0:
        ax.set_ylabel(r'$P_{\rm gg}(k) \, \left[{\rm Mpc}^3\right]$')
    else:
        ax.set_yticklabels([])
 
    if ind1==1:
        ax.set_xlabel(r'$k \, \left[{\rm Mpc}^{-1}\right]$')
    else:
        ax.set_xticklabels([])
        
    ax.set_xticks([0.01, 1])
    ax.set_xticklabels(['0.01', '1'])
    
    ax.text(0.4, 3e4, r'$\mu=%.2f$'%(mu), size=12, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))

fig.legend(prop={'size':14}, ncol=3, loc='upper center')
fig.subplots_adjust(wspace=0.01, hspace=0.01)

The different amplitude as a function of the value of $\mu$ highlights the anisotropic signal introduced by redshift-space distortions, increasing towards higher values of $\mu$, i.e. along the line of sight.

### 4) Plotting the galaxy power spectrum multipoles $P^{(\ell)}(k,z)$ (i.e. the main deliverable of the `Spectro` class)

Now, we check directly the galaxy power spectrum multipoles using the `Spectro` class. We could have used the instance of `Spectro` that is already contained as an attribute of `like`, similarly to what we did for the instance of `Cosmology`, but here we want to show how to use the `Spectro` class from the very beginning.

First, we create three separate instances of `Spectro`, one for each of the cases defined above. The `Spectro` class requires two arguments to be instantiated. The first one is a cosmo dictionary, with the internal interpolators already up to date. The second is a list of spectroscopic bin centers, to correct for the purity of the sample. Here we pass respectively the three deep copies of the cosmo dictionary defined above, and the values of the bin centers defined in the info dictionary.

In [ ]:
from cloe.spectroscopic_survey.spectro import Spectro

spec_ins_0 = Spectro(cosmo_dic_0, info['likelihood']['Euclid']['data']['spectro']['redshifts'])
spec_ins_1 = Spectro(cosmo_dic_1, info['likelihood']['Euclid']['data']['spectro']['redshifts'])
spec_ins_2 = Spectro(cosmo_dic_2, info['likelihood']['Euclid']['data']['spectro']['redshifts'])

Then we call the method `multipole_spectra` of the `Spectro` class to compute the multipoles of order 0,2,4 (i.e. monopole, quadrupole, and hexadecapole).

In [ ]:
mps_0_0 = np.empty(len(ks))
mps_0_2 = np.empty(len(ks))
mps_0_4 = np.empty(len(ks))
for i in range(len(ks)):
    mps_0_0[i], mps_0_2[i], mps_0_4[i] = spec_ins_0.multipole_spectra(z, ks[i], ms=[0,2,4])
    
mps_1_0 = np.empty(len(ks))
mps_1_2 = np.empty(len(ks))
mps_1_4 = np.empty(len(ks))
for i in range(len(ks)):
    mps_1_0[i], mps_1_2[i], mps_1_4[i] = spec_ins_1.multipole_spectra(z, ks[i], ms=[0,2,4])
    
mps_2_0 = np.empty(len(ks))
mps_2_2 = np.empty(len(ks))
mps_2_4 = np.empty(len(ks))
for i in range(len(ks)):
    mps_2_0[i], mps_2_2[i], mps_2_4[i] = spec_ins_2.multipole_spectra(z, ks[i], ms=[0,2,4])

Finally, we plot the multipoles.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12,4))

ax = axs[0]
ax.grid(ls='--', lw=0.3)
ax.loglog(ks, mps_0_0, lw=3, label=r'$\ell=0$')
ax.loglog(ks, mps_0_2, lw=3, label=r'$\ell=2$')
ax.loglog(ks, mps_0_4, lw=3, label=r'$\ell=4$')
ax.set_ylim([0.01, 3e5])
ax.set_xticks([0.01, 1])
ax.set_xticklabels(['0.01', '1'])
ax.set_ylabel(r'$P_{\rm gg}^{(\ell)}(k) \, \left[{\rm Mpc}^3\right]$')
ax.set_xlabel(r'$k \, \left[{\rm Mpc}^{-1}\right]$')
ax.text(0.001, 0.03, 'Linear theory', size=14, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))

ax = axs[1]
ax.grid(ls='--', lw=0.3)
ax.loglog(ks, mps_1_0, lw=3)
ax.loglog(ks, mps_1_2, lw=3)
ax.loglog(ks, mps_1_4, lw=3)
ax.set_ylim([0.01, 3e5])
ax.set_xticks([0.01, 1])
ax.set_xticklabels(['0.01', '1'])
ax.set_yticklabels([])
ax.set_xlabel(r'$k \, \left[{\rm Mpc}^{-1}\right]$')
ax.text(0.001, 0.03, 'EFT (all params = 0)', size=14, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))

ax = axs[2]
ax.grid(ls='--', lw=0.3)
ax.loglog(ks, mps_2_0, lw=3)
ax.loglog(ks, mps_2_2, lw=3)
ax.loglog(ks, mps_2_4, lw=3)
ax.set_ylim([0.01, 3e5])
ax.set_xticks([0.01, 1])
ax.set_xticklabels(['0.01', '1'])
ax.set_yticklabels([])
ax.set_xlabel(r'$k \, \left[{\rm Mpc}^{-1}\right]$')
ax.text(0.001, 0.03, 'EFT', size=14, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))

fig.legend(prop={'size':14}, ncol=3, loc='upper center')
fig.subplots_adjust(wspace=0.01, hspace=0.01)

### 5) Timing different modelling options for GCspectro

Now, we measure the speed for an evaluation of the likelihood when only considering the GCspectro probe. At the moment we only test the linear case (`NL_flag_spectro=0`) and the EFT case (`NL_flag_spectro=1`).

We create a copy of the info dictionary and modify some of its arguments, i.e. the nonlinear flag, and the parameter $n_{\rm s}$ (since we want to sample over it). Then we create an instance of a Cobaya model, as we did in the previous cells, and we measure the speed for the evaluation of the likelihood.

In [ ]:
# First copy the info dict 
info_speed = info.copy()

# We measure the time it takes for a posterior evaluation. Arbitrarily, we choose to sample ns.
info_speed['params']['ns'] = {'prior':{'min':0.95, 'max':0.97}}

# We start with NL_flag_phot_matter = 0
info_speed['likelihood']['Euclid']['NL_flag_spectro'] = 0

model_speed = get_model(info_speed)

# Evaluate posterior once to ensure that everything is loaded before measuring time
point = dict(zip(model_speed.parameterization.sampled_params(),
                 model_speed.prior.sample(ignore_external=True)[0]))
logposterior = model_speed.logposterior(point)

# Evalute posterior n_eval times and average

n_eval = 10

t1 = time.time()

for i in range(n_eval):
    point = dict(zip(model_speed.parameterization.sampled_params(),
                     model_speed.prior.sample(ignore_external=True)[0]))

    logposterior = model_speed.logposterior(point)

t2 = time.time()

print("\nAverage time for posterior evaluation for NL_spectro =",info_speed['likelihood']['Euclid']['NL_flag_spectro'],
      "is",(t2-t1)/n_eval,"seconds.")

In [ ]:
# We start with NL_flag_phot_matter = 0
info_speed['likelihood']['Euclid']['NL_flag_spectro'] = 1

model_speed = get_model(info_speed)

# Evaluate posterior once to ensure that everything is loaded before measuring time
point = dict(zip(model_speed.parameterization.sampled_params(),
                 model_speed.prior.sample(ignore_external=True)[0]))
logposterior = model_speed.logposterior(point)

# Evalute posterior n_eval times and average

n_eval = 10

t1 = time.time()

for i in range(n_eval):
    point = dict(zip(model_speed.parameterization.sampled_params(),
                     model_speed.prior.sample(ignore_external=True)[0]))

    logposterior = model_speed.logposterior(point)

t2 = time.time()

print("\nAverage time for posterior evaluation for NL_spectro =",info_speed['likelihood']['Euclid']['NL_flag_spectro'],
      "is",(t2-t1)/n_eval,"seconds.")

**Please note:** similarly to the evaluation of the likelihood that happens at the beginning of this notebook, the time provided here also includes an extra contribution due to the initialisation of the `Photo` class.